In [ ]:
import sys
sys.path.insert(0, r'C:\Users\ideapad\Desktop\Repositório Final Tese\Recommendation System (Draco)\draco2')

import altair as alt
import warnings
import pandas as pd
from draco import dict_union, dict_to_facts, Draco
from draco.renderer import AltairRenderer
from IPython.display import display
import pprint as pp
import json

warnings.filterwarnings("ignore")

dataset = pd.read_csv(r"C:\Users\ideapad\Desktop\Repositório Final Tese\Recommendation System (Draco)\testing_datasets\Superstore.csv", encoding="ISO-8859-1")
datasetFiltered = dataset[dataset[['Rowid',"Orderid","Orderdate","Shipdate","Shipmode","Customerid","Customername","Segment","Country","City","State","Postalcode","Region","Productid","Category","Subcategory","Productname","Sales","Quantity","Discount","Profit"]].notnull().all(1)]

datasetFiltered["Orderdate"] = pd.to_datetime(datasetFiltered["Orderdate"])
#print(datasetFiltered['Orderdate'].dtype)
datasetFiltered['MonthYear'] = pd.to_datetime(datasetFiltered['Orderdate']).dt.strftime('%Y-%m')
datasetFiltered['Month'] = datasetFiltered["Orderdate"].dt.month_name().str.slice(stop=3)
#print(datasetFiltered['MonthYear'])

d = Draco()
positionList = list()
renderer = AltairRenderer(concat_mode="hconcat")


def show(*args, df: pd.DataFrame = datasetFiltered):
    spec = dict_union(*args)
    prog = dict_to_facts(spec)
    if not d.check_spec(prog):
        print("\n".join(prog))
        print(d.get_violations(prog))
        assert False, "Invalid spec"

    # Display the rendered VL chart and the ASP
    chart = renderer.render(spec, df)
    # Add to the list (The Ranking List)
    positionList.append(prog)
    #print(json.dumps(spec, indent=2))
    display(chart)
    display(prog)

def pairCreator(positive,negative):
    x = '{"task": "compare"}'
    z = json.loads(x)
    z.update({"positive":positive})
    z.update({"negative":negative})
    #print(json.dumps(z))
    return json.dumps(z)

def jsonCreator(allList):
    output = ""
    for x in range(len(allList)-1):
        if (x == 0):
            output = str(pairCreator(positionList[x],positionList[x+1]))
        else:
            output = output + ", " + str(pairCreator(positionList[x],positionList[x+1]))
    return output

def data(fields: list[str], df: pd.DataFrame = datasetFiltered) -> dict:
    number_rows, _ = df.shape
    return {
        "number_rows": number_rows,
        "field": [
            x
            for x in [
                {"name": "Rowid", "type": "string", "unique": str(datasetFiltered["Rowid"].nunique()), "__id__": "Rowid"},
                {"name": "Orderid", "type": "string", "unique": str(datasetFiltered["Orderid"].nunique()), "__id__": "Orderid"},
                {"name": "Orderdate", "type": "datetime", "unique": str(datasetFiltered["Orderdate"].nunique()), "__id__": "Orderdate"},
                {"name": "Shipdate", "type": "datetime", "unique": str(datasetFiltered["Shipdate"].nunique()), "__id__": "Shipdate"},
                {"name": "Shipmode", "type": "string", "unique": str(datasetFiltered["Shipmode"].nunique()), "__id__": "Shipmode"},
                {"name": "Customerid", "type": "string", "unique": str(datasetFiltered["Customerid"].nunique()), "__id__": "Customerid"},
                {"name": "Customername", "type": "string", "unique": str(datasetFiltered["Customername"].nunique()), "__id__": "Customername"},
                {"name": "Segment", "type": "string", "unique": str(datasetFiltered["Segment"].nunique()), "__id__": "Segment"},
                {"name": "Country", "type": "string", "unique": str(datasetFiltered["Country"].nunique()), "__id__": "Country"},
                {"name": "City", "type": "string", "unique": str(datasetFiltered["City"].nunique()), "__id__": "City"},
                {"name": "State", "type": "string", "unique": str(datasetFiltered["State"].nunique()), "__id__": "State"},
                {"name": "Postalcode", "type": "number", "unique": str(datasetFiltered["Postalcode"].nunique()), "__id__": "Postalcode"},
                {"name": "Region", "type": "string", "unique": str(datasetFiltered["Region"].nunique()), "__id__": "Region"},
                {"name": "Productid", "type": "string", "unique": str(datasetFiltered["Productid"].nunique()), "__id__": "Productid"},
                {"name": "Category", "type": "string", "unique": str(datasetFiltered["Category"].nunique()), "__id__": "Category"},
                {"name": "Subcategory", "type": "string", "unique": str(datasetFiltered["Subcategory"].nunique()), "__id__": "Subcategory"},
                {"name": "Productname", "type": "string", "unique": str(datasetFiltered["Productname"].nunique()), "__id__": "Productname"},
                {"name": "Sales", "type": "number", "unique": str(datasetFiltered["Sales"].nunique()), "__id__": "Sales"},
                {"name": "Quantity", "type": "number", "unique": str(datasetFiltered["Quantity"].nunique()), "__id__": "Quantity"},
                {"name": "Discount", "type": "number", "unique": str(datasetFiltered["Discount"].nunique()), "__id__": "Discount"},
                {"name": "Profit", "type": "number", "unique": str(datasetFiltered["Profit"].nunique()), "__id__": "Profit"},
                {"name": "Month", "type": "string", "unique": str(datasetFiltered["Month"].nunique()), "__id__": "Month"},
                {"name": "MonthYear", "type": "datetime", "unique": str(datasetFiltered["MonthYear"].nunique()), "__id__": "Monthyear"}
            ]
            if x["name"] in fields
        ],
    }

# -- Interpretação --

## Distribution

###  -> Características
- Desconto (Métrica)
- Categoria (Dimensão)
- ?????? (Agregação)
- ?????? (Tempo)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"},
                            {"channel": "color", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "color", "type": "linear"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"},
                            {"channel": "color", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "color", "type": "linear"},
                    {"channel": "text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:

show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"},
                            {"channel": "color", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel": "color", "type": "linear"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"},
                            {"channel": "color", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel": "color", "type": "linear"},
                    {"channel": "text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "line",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "line",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "line",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "line",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "area",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "area",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "area",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [
                            {"channel": "y", "field": "Category"},
                            {"channel": "x", "field": "Discount", "aggregate": "mean"}
                        ],
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "field": "Discount", "aggregate": "mean"}
                        ]
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel":"text", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "polar",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {
                                "channel": "y",
                                "field": "Discount",
                                "aggregate": "sum",
                                "stack": "zero",
                            },
                            {"channel": "color", "field": "Category"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "color", "type": "categorical"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "coordinates": "polar",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {
                                "channel": "y",
                                "field": "Discount",
                                "aggregate": "mean",
                            },
                        ],
                    },
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Discount", "Category"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "rect",
                        "encoding": [
                            {"channel": "x", "field": "Discount"},
                            {"channel": "y", "field": "Category"},
                            {"channel": "color", "field": "Discount"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "color", "type": "categorical"}
                ],
            }
        ]
    },
)

In [ ]:
pairCreator(positionList[0],positionList[1])